<a href="https://colab.research.google.com/github/urenajose/DS-Unit-2-Regression-1/blob/master/Jose_Urena_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [46]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [47]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [48]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [49]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [50]:
import datetime

In [51]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23040 entries, 0 to 23039
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         23040 non-null  object 
 1   NEIGHBORHOOD                    23040 non-null  object 
 2   BUILDING_CLASS_CATEGORY         23040 non-null  object 
 3   TAX_CLASS_AT_PRESENT            23039 non-null  object 
 4   BLOCK                           23040 non-null  int64  
 5   LOT                             23040 non-null  int64  
 6   EASE-MENT                       0 non-null      float64
 7   BUILDING_CLASS_AT_PRESENT       23039 non-null  object 
 8   ADDRESS                         23040 non-null  object 
 9   APARTMENT_NUMBER                5201 non-null   object 
 10  ZIP_CODE                        23039 non-null  float64
 11  RESIDENTIAL_UNITS               23039 non-null  float64
 12  COMMERCIAL_UNITS                

In [53]:
df['SALE_PRICE'].value_counts(), "to check value counts for SALE_PRICE, I noticed some extrem low values"

(0          6909
 10          199
 800000      125
 750000      121
 650000      120
            ... 
 5236177       1
 229000        1
 397218        1
 4112000       1
 1751425       1
 Name: SALE_PRICE, Length: 3831, dtype: int64,
 'to check value counts for SALE_PRICE, I noticed some extrem low values')

In [54]:
df = df[(df['BUILDING_CLASS_CATEGORY'] !='01 ONE FAMILY DWELLINGS') & (df['SALE_PRICE'] > 11)]

In [55]:
df.shape

(12590, 21)

In [56]:
df[['SALE_DATE',"YEAR_BUILT"]]

,SALE_DATE,YEAR_BUILT
65,01/02/2019,2013.0
66,01/02/2019,1960.0
67,01/02/2019,1925.0
70,01/02/2019,1901.0
71,01/02/2019,2004.0
...,...,...
23034,04/30/2019,1945.0
23036,04/30/2019,1929.0
23037,04/30/2019,1932.0
23038,04/30/2019,1931.0


In [57]:
df['SALE_DATE'] = df['SALE_DATE'].astype('datetime64[ns]')

In [58]:
df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].fillna("0")

In [59]:
# df[(df['LAND_SQUARE_FEET'])=="x"] "just to check"

In [60]:
df['LAND_SQUARE_FEET'] = [x.replace(",","") for x in df['LAND_SQUARE_FEET']]
# df['LAND_SQUARE_FEET'] = [x.replace("x",'0') for x in df['LAND_SQUARE_FEET']]

In [61]:
df['LAND_SQUARE_FEET'].astype(float,copy=True), 'getting a error will come back to it'

(65       18975.0
 66           0.0
 67           0.0
 70       28335.0
 71           0.0
           ...   
 23034     4000.0
 23036        0.0
 23037        0.0
 23038     2078.0
 23039    15151.0
 Name: LAND_SQUARE_FEET, Length: 12590, dtype: float64,
 'getting a error will come back to it')

In [62]:
# [x for x in (x.replace(",","") for x in df['LAND_SQUARE_FEET']) if x is not None] 'did not work'

In [63]:
df['LAND_SQUARE_FEET']

65       18975
66           0
67           0
70       28335
71           0
         ...  
23034     4000
23036        0
23037        0
23038     2078
23039    15151
Name: LAND_SQUARE_FEET, Length: 12590, dtype: object

In [64]:
display(df.isnull().sum())
print(df.shape)

BOROUGH                               0
NEIGHBORHOOD                          0
BUILDING_CLASS_CATEGORY               0
TAX_CLASS_AT_PRESENT                  1
BLOCK                                 0
LOT                                   0
EASE-MENT                         12590
BUILDING_CLASS_AT_PRESENT             1
ADDRESS                               0
APARTMENT_NUMBER                   8710
ZIP_CODE                              1
RESIDENTIAL_UNITS                     1
COMMERCIAL_UNITS                      1
TOTAL_UNITS                           1
LAND_SQUARE_FEET                      0
GROSS_SQUARE_FEET                     1
YEAR_BUILT                           32
TAX_CLASS_AT_TIME_OF_SALE             0
BUILDING_CLASS_AT_TIME_OF_SALE        0
SALE_PRICE                            0
SALE_DATE                             0
dtype: int64

(12590, 21)


##One-hot encoding

### feature selection

In [65]:
datatype = df.dtypes #pass dtypes of the df to variable

In [66]:
datatype_cat = datatype[(datatype == 'object') | (datatype == 'category')].index.tolist() #return the columns with categorical and object
datatype_cat

['BOROUGH',
 'NEIGHBORHOOD',
 'BUILDING_CLASS_CATEGORY',
 'TAX_CLASS_AT_PRESENT',
 'BUILDING_CLASS_AT_PRESENT',
 'ADDRESS',
 'APARTMENT_NUMBER',
 'LAND_SQUARE_FEET',
 'BUILDING_CLASS_AT_TIME_OF_SALE']

In [67]:
for x in datatype_cat: #for loop to run value_counts to select features with low Cardinality, and drop Hight Cardinality features
  print(f'{df[x].value_counts()}\n')
  print(f'count: {len(df[x].value_counts())}\n')

3    3704
1    3687
4    3385
2    1265
5     549
Name: BOROUGH, dtype: int64

count: 5

OTHER                      10190
UPPER EAST SIDE (59-79)      381
UPPER EAST SIDE (79-96)      338
FLUSHING-NORTH               330
UPPER WEST SIDE (59-79)      259
BEDFORD STUYVESANT           256
FOREST HILLS                 205
ASTORIA                      174
GRAMERCY                     159
EAST NEW YORK                153
BOROUGH PARK                 145
Name: NEIGHBORHOOD, dtype: int64

count: 11

10 COOPS - ELEVATOR APARTMENTS                3042
02 TWO FAMILY DWELLINGS                       2658
13 CONDOS - ELEVATOR APARTMENTS               2462
03 THREE FAMILY DWELLINGS                      751
09 COOPS - WALKUP APARTMENTS                   612
07 RENTALS - WALKUP APARTMENTS                 413
15 CONDOS - 2-10 UNIT RESIDENTIAL              346
04 TAX CLASS 1 CONDOS                          308
44 CONDO PARKING                               288
17 CONDO COOPS                              

#droping columns

In [68]:
df["LAND_SQUARE_FEET"].value_counts(), '''even thought LAND_SQUARE_FEEt was int64 the one-hot is converting the index into columns, 
 resulting in shape of ((9600, 1870), (2990, 1870))'''

(0       6266
 2500     450
 2000     403
 4000     230
 3000     122
         ... 
 1460       1
 4608       1
 6454       1
 2481       1
 3417       1
 Name: LAND_SQUARE_FEET, Length: 2182, dtype: int64,
 'even thought LAND_SQUARE_FEEt was int64 the one-hot is converting the index into columns, \n resulting in shape of ((9600, 1870), (2990, 1870))')

In [69]:
df_nc= df.drop(['APARTMENT_NUMBER','EASE-MENT',
 'BUILDING_CLASS_CATEGORY',
 'BUILDING_CLASS_AT_PRESENT',
 'ADDRESS',
 'APARTMENT_NUMBER',
 'BUILDING_CLASS_AT_TIME_OF_SALE','LAND_SQUARE_FEET'],axis=1)

In [70]:
df_nc.dropna(inplace=True)

## Train & Test

In [71]:
train = df_nc[(df_nc['SALE_DATE'] < datetime.datetime(2019,4,1) )]
test = df_nc[(df_nc['SALE_DATE'] > datetime.datetime(2019,3,31) )]

In [72]:
train1 = train.drop('SALE_PRICE',axis=1)
test1 = test.drop('SALE_PRICE',axis=1)

###Train

In [73]:
print(train1.shape)
display(train1.head())
display(train1.tail())

(9595, 13)


,BOROUGH,NEIGHBORHOOD,TAX_CLASS_AT_PRESENT,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_DATE
65,1,OTHER,2,691,1130,10011.0,1.0,0.0,1.0,1077.0,2013.0,2,2019-01-02
66,1,OTHER,2,1347,18,10022.0,0.0,0.0,0.0,0.0,1960.0,2,2019-01-02
67,1,UPPER EAST SIDE (79-96),2,1491,62,10075.0,0.0,0.0,0.0,0.0,1925.0,2,2019-01-02
70,1,UPPER WEST SIDE (59-79),2,1167,1272,10023.0,1.0,0.0,1.0,545.0,1901.0,2,2019-01-02
71,1,UPPER WEST SIDE (59-79),2,1171,2200,10069.0,1.0,0.0,1.0,827.0,2004.0,2,2019-01-02


,BOROUGH,NEIGHBORHOOD,TAX_CLASS_AT_PRESENT,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_DATE
18121,5,OTHER,1,5378,13,10312.0,1.0,0.0,1.0,2833.0,2018.0,1,2019-03-29
18122,5,OTHER,2,3585,1246,10306.0,1.0,0.0,1.0,850.0,1988.0,2,2019-03-29
18126,5,OTHER,1,4500,167,10308.0,2.0,0.0,2.0,2920.0,1984.0,1,2019-03-29
18138,5,OTHER,1,236,88,10310.0,2.0,0.0,2.0,2048.0,1920.0,1,2019-03-29
18144,2,OTHER,1,3812,52,10462.0,2.0,0.0,2.0,2982.0,1920.0,1,2019-03-30


###Test

In [74]:
print(test1.shape)
display(test1.head())
display(test1.tail())

(2963, 13)


,BOROUGH,NEIGHBORHOOD,TAX_CLASS_AT_PRESENT,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_DATE
18167,1,OTHER,2,695,1562,10011.0,1.0,0.0,1.0,670.0,2005.0,2,2019-04-01
18168,1,OTHER,2,767,1609,10011.0,1.0,0.0,13.0,36835.0,1912.0,2,2019-04-01
18169,1,OTHER,2,767,1612,10011.0,1.0,0.0,13.0,36835.0,1912.0,2,2019-04-01
18170,1,OTHER,2,721,1008,10011.0,0.0,0.0,0.0,0.0,1932.0,2,2019-04-01
18172,1,OTHER,2C,145,1203,10013.0,1.0,0.0,1.0,1297.0,1920.0,2,2019-04-01


,BOROUGH,NEIGHBORHOOD,TAX_CLASS_AT_PRESENT,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_DATE
23034,4,OTHER,1,12536,38,11434.0,2.0,0.0,2.0,2136.0,1945.0,1,2019-04-30
23036,4,OTHER,2,169,29,11104.0,0.0,0.0,0.0,0.0,1929.0,2,2019-04-30
23037,4,OTHER,2,131,4,11377.0,0.0,0.0,0.0,0.0,1932.0,2,2019-04-30
23038,4,OTHER,1,8932,18,11421.0,2.0,1.0,3.0,2200.0,1931.0,1,2019-04-30
23039,4,OTHER,2,1216,1161,11377.0,1.0,0.0,85.0,854.0,1927.0,2,2019-04-30


*italicized text*##One hot

In [75]:
import category_encoders as ce

In [76]:
encoder = ce.OneHotEncoder(use_cat_names=True)

In [77]:
x_train = encoder.fit_transform(train1)

In [78]:
x_test = encoder.transform(test1)

In [79]:
x_train.shape, x_test.shape

((9595, 35), (2963, 35))

In [80]:
# x_train.drop("SALE_DATE",axis=1,inplace=True)

In [81]:
# x_test.drop("SALE_DATE",axis=1,inplace=True)

In [82]:
y_train = train['SALE_PRICE']

In [123]:
y_test = test['SALE_PRICE']

In [83]:
from sklearn.feature_selection import SelectKBest, f_regression

In [84]:
selector = SelectKBest(score_func=f_regression, k=13)

In [86]:
x_train_selected = selector.fit_transform(x_train.drop('SALE_DATE',axis=1),y_train)

In [87]:
x_test_selected = selector.transform(x_test.drop('SALE_DATE',axis=1))

In [97]:
print(f'''x_train_selected:
\n 
{x_train_selected}, 
\n
x_test_selected:
\n
{x_train_selected}''')

x_train_selected:

 
[[1.000e+00 0.000e+00 0.000e+00 ... 1.000e+00 1.077e+03 2.000e+00]
 [1.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 2.000e+00]
 [1.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 2.000e+00]
 ...
 [0.000e+00 0.000e+00 0.000e+00 ... 2.000e+00 2.920e+03 1.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 ... 2.000e+00 2.048e+03 1.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 ... 2.000e+00 2.982e+03 1.000e+00]], 


x_test_selected:


[[1.000e+00 0.000e+00 0.000e+00 ... 1.000e+00 1.077e+03 2.000e+00]
 [1.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 2.000e+00]
 [1.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 2.000e+00]
 ...
 [0.000e+00 0.000e+00 0.000e+00 ... 2.000e+00 2.920e+03 1.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 ... 2.000e+00 2.048e+03 1.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 ... 2.000e+00 2.982e+03 1.000e+00]]


In [104]:
selected_mask = selector.get_support()
all_names = x_train.drop('SALE_DATE',axis=1).columns
selected_names = all_names[~selected_mask]

print("Feature selected")
feature_selected = []
for name in selected_names:
  feature_selected.append(name)
print(feature_selected)

Feature selected
['BOROUGH_2', 'BOROUGH_5', 'NEIGHBORHOOD_OTHER', 'NEIGHBORHOOD_UPPER EAST SIDE (79-96)', 'NEIGHBORHOOD_UPPER WEST SIDE (59-79)', 'NEIGHBORHOOD_BEDFORD STUYVESANT', 'NEIGHBORHOOD_EAST NEW YORK', 'NEIGHBORHOOD_ASTORIA', 'NEIGHBORHOOD_FLUSHING-NORTH', 'NEIGHBORHOOD_GRAMERCY', 'NEIGHBORHOOD_UPPER EAST SIDE (59-79)', 'NEIGHBORHOOD_BOROUGH PARK', 'NEIGHBORHOOD_FOREST HILLS', 'TAX_CLASS_AT_PRESENT_2B', 'TAX_CLASS_AT_PRESENT_2A', 'TAX_CLASS_AT_PRESENT_1B', 'TAX_CLASS_AT_PRESENT_1A', 'TAX_CLASS_AT_PRESENT_2C', 'TAX_CLASS_AT_PRESENT_1C', 'LOT', 'YEAR_BUILT']


In [260]:
from sklearn.linear_model import Ridge

In [397]:
my_ridge = Ridge(alpha=10,normalize=True)

In [398]:
my_ridge.fit(x_train.drop("SALE_DATE",axis=1), y_train)

Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None, normalize=True,
      random_state=None, solver='auto', tol=0.001)

In [399]:
y_pred = my_ridge.predict(x_test.drop("SALE_DATE",axis=1))
y_pred

array([2202714.10558275, 4244136.8848287 , 4244094.23456456, ...,
       1789340.51778414, 1648460.54793516, 4274458.53487181])

In [400]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [401]:
r2_score(y_test, y_pred)

0.09825411586535271

In [402]:
mae_ridge = mean_absolute_error(y_test, y_pred)
mae_ridge

2609597.1946393712

In [403]:
test['SALE_PRICE'].mean(), train['SALE_PRICE'].mean()

(2672575.98481269, 2109090.015737363)

In [404]:
x_train1 = x_train[feature_selected]
x_test1 = x_test[feature_selected]

In [440]:
my_ridge = Ridge(alpha=.1,normalize=True)

In [441]:
my_ridge.fit(x_train1, y_train)

Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None, normalize=True,
      random_state=None, solver='auto', tol=0.001)

In [442]:
y_pred = my_ridge.predict(x_test1)
y_pred

array([2493901.8779403 , 2490123.89130221, 2490123.79169101, ...,
       2490989.31792879, 2490948.24634469, 2490747.86755188])

In [444]:
mae_ridge = mean_absolute_error(y_test, y_pred)
mae_ridge

2784735.541004641

In [443]:
r2_score(y_test, y_pred)

0.0038464024541589037